<a href="https://colab.research.google.com/github/khchoi-physik/pbh_simulations/blob/main/GRFSimulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Import packages

from scipy.special import hankel2  # kurtosis -3 convention
from scipy.stats import skew, kurtosis

import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
import os
import gc
import time as ti

In [ ]:
# @title 1.1 Random field simulations

class grf_sim:

    def __init__(self, pixel=2**9, mean=0, std_dev=1):
        self.pixel = pixel
        self.mean = mean
        self.std_dev = std_dev
        self.fft_white_noise, self.k_norm = self.grf_initialization()

    def grf_initialization(self):
        # Guassian white noise generation from GPU
        white_noise = cp.random.normal(self.mean, self.std_dev, (self.pixel, self.pixel, self.pixel))

        # FFT of Guassian white noise
        fft_white_noise = cp.fft.fftn(white_noise).astype(cp.complex128)
        del white_noise

        # Generating FFT momentum
        kx = cp.fft.fftfreq(self.pixel)*self.pixel
        ky = cp.fft.fftfreq(self.pixel)*self.pixel
        kz = cp.fft.fftfreq(self.pixel)*self.pixel

        # Genearting FFT momentum 3D array
        kx_grid, ky_grid, kz_grid = cp.meshgrid(kx, ky, kz)
        # k_grid = np.meshgrid(kx, ky, kz)
        del kx, ky, kz

        # Norm of k
        k_norm =  cp.sqrt( kx_grid**2 + ky_grid**2 + kz_grid**2 ) # k = sqrt( kx^2 + ky^2  + kz^2 )
        k_norm[0,0,0] = cp.inf # Regularize divergence at k=0, for any power spectrum of spectral index less than.
        del kx_grid, ky_grid, kz_grid

        return fft_white_noise, k_norm

    def generate_grf_desitter(self, time=1, amplitude=1/10, n_index=1.5):

        k_norm = cp.asnumpy(self.k_norm)
        amplitude = cp.asnumpy(amplitude)

        fourier_amplitudes_sqrt =  np.sqrt(np.pi) * 0.5 *  amplitude   *  (time**(3/2)) * hankel2(n_index, time * ( (2 * np.pi/self.pixel)* k_norm) )
        #(2 * np.pi / pixel) *
        del k_norm

        fourier_amplitudes_sqrt[0,0,0] = 0 # Regularize divergences at k_norm = 0.
        fourier_amplitudes_sqrt = cp.asarray(fourier_amplitudes_sqrt).astype(cp.complex128)
        fourier_amplitudes_sqrt = cp.abs(fourier_amplitudes_sqrt).astype(cp.complex128)

        fourier_amplitudes_sqrt *= self.fft_white_noise

        gaussian_random_field = cp.fft.ifftn(fourier_amplitudes_sqrt)
        del fourier_amplitudes_sqrt

        return gaussian_random_field.real


    def run_simulation(self,  start_time, stop_time, time_step, run_id=0,  amplitude = 1/10, n_index=1.5, extra_time_range = [], save_grf_time = [], plot_bool = False ):

        t0 = ti.time()

        num_steps = int( (stop_time - start_time) / time_step) + 1

        time_line = np.linspace(start_time, stop_time, num_steps)
        time_line = np.concatenate( (time_line,  np.array(extra_time_range) ) )

        statistics  = {'std': [], 'mean': [], 'skew': [], 'kurt': [], 'max_val': [], 'over_2std': []}

        for time in time_line:

            t1 = ti.time()
            grf = self.generate_grf_desitter(time=time, amplitude=amplitude, n_index=n_index)
            grf = grf.real

            # Gather statistics
            stat_fig, grf_std, grf_mean, grf_skew, grf_kurt = self.stats_overview(grf)

            grf_abs_flatten = cp.abs(grf).flatten()
            grf_max_val = cp.max(grf_abs_flatten).get()
            samples_over_2std = grf_abs_flatten[grf_abs_flatten > (2 * grf_std)]
            percent_over_2std = (samples_over_2std.size / grf_abs_flatten.size) * 100

            for key, value in zip(statistics.keys(), [grf_std, grf_mean, grf_skew, grf_kurt, grf_max_val, percent_over_2std]):
                statistics[key].append([time, value])

            if run_id == 0 and plot_bool == True and round(time,4) in save_grf_time:
                cp.save(f'grf_t={time:.4f}_pixel={self.pixel}.npy', grf)
            #   plot_temperature_maps(grf, time, pixel, z_pos, grf_std, grf_mean, plot_bool, stat_fig, n_sigmas, vmin, vmax)

                tf = ti.time() - t1
                print(f't= {time:.4f}: Computational time is: {tf:.2f} seconds')
                plt.close('all')

        # Save statistics
        self.save_statistics(statistics, run_id)

        tf = ti.time() - t0
        print(f'Run: {run_id+1}: Computational time for this runs is: {tf:.2f} seconds')
        plt.close('all')


    def stats_overview(self, grf):

        all_points = cp.asnumpy(grf.flatten())
        del grf

        grf_std = np.std(all_points)
        grf_mean = np.mean(all_points)
        grf_skew = skew(all_points)
        grf_kurt = kurtosis(all_points)

        plt.hist(all_points, bins=256)

        textstr = (f'Std. Dev. = {grf_std:.4f}\n'
                f'Mean = {grf_mean:.4f}\n'
                f'Skewness = {grf_skew:.4f}\n'
                f'Kurtosis = {grf_kurt:.4f}')

        plt.gca().text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=14,
                    verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))


        plt.title('Random Field Statistics', fontsize=16)
        plt.xlabel('Field amplitude', fontsize=16)
        plt.ylabel('Number of data points', fontsize=16)

        return plt.gcf(), grf_std, grf_mean, grf_skew, grf_kurt


    def save_statistics(self, statistics, run_id):
        np.save(f'percent_over_2std_array_run_{run_id+1}.npy', np.array(statistics['over_2std']))
        np.save(f'max_val_array_run_{run_id+1}.npy', np.array(statistics['max_val']) )
        np.save(f'std_array_run_{run_id+1}.npy', np.array(statistics['std']))
        np.save(f'mean_array_run_{run_id+1}.npy', np.array(statistics['mean']))
        np.save(f'skew_array_run_{run_id+1}.npy', np.array(statistics['skew']))
        np.save(f'kurt_array_run_{run_id+1}.npy', np.array(statistics['kurt']))